In [1]:
import torch  # PyTorch, needless to say ;)
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig # Main components of LLM

### Check if CUDA is available

In [2]:
torch.cuda.is_available()

True

### Quantization config

In [3]:
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

## Load Model

In [4]:
model_id = "memma-1-2b-cmb-seq2seq-hf"

# Change `local_files_only` to `False` to download from HuggingFace Hub.
tokenizer = AutoTokenizer.from_pretrained(model_id, local_files_only=True)
model = AutoModelForCausalLM.from_pretrained(model_id, device_map={"":0}, quantization_config=bnb_config, local_files_only=True)

Gemma's activation function should be approximate GeLU and not exact GeLU.
Changing the activation function to `gelu_pytorch_tanh`.if you want to use the legacy `gelu`, edit the `model.config` to set `hidden_activation=gelu`   instead of `hidden_act`. See https://github.com/huggingface/transformers/pull/29402 for more details.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

## Inference

In [17]:
text = "解師傅 "

device = "cuda:0"
input_prompt = tokenizer(text, return_tensors="pt").to(device)
outputs = model.generate(**input_prompt, max_new_tokens=200, do_sample=True, top_k=60, top_p=0.9, repetition_penalty=1.2, temperature=0.3)
print(tokenizer.decode(outputs[0], skip_special_tokens=True))

解師傅 你的刀工太差了，你還是21小時後再來吧
